In [1]:
import numpy as np
import pandas as pd

### dataset, cur_fold, r, lr, num_iter= sys.argv[1:]


In [2]:
err = {}
for dataset in [1, 2]:
    err[dataset] = {}
    for cur_fold in range(5):
        err[dataset][cur_fold] = {}
        for r in range(1 ,21):
            err[dataset][cur_fold][r] = {}
            for lr in [0.01, 0.1, 1.0, 2.0]:
                err[dataset][cur_fold][r][lr] = {}
                for num_iter in range(100, 2600, 400):
                    try:
                        err[dataset][cur_fold][r][lr][num_iter] = \
                        np.load("../code/baseline/mtf-nested/mtf-error-{}-{}-{}-{}-{}.npy".format(dataset, cur_fold, r, lr, num_iter)).item()
                    except:
                        print (dataset, cur_fold, r, lr, num_iter)

In [3]:
iter = pd.DataFrame(err[1][0][3][0.01]).mean().idxmin()

In [4]:
pd.DataFrame(err[1][0][3][0.01]).mean()[2500]

174.27279032501659

In [5]:
best_r = {}
best_lr = {}
best_iter = {}
for dataset in [1, 2]:
    best_r[dataset] = {}
    best_lr[dataset] = {}
    best_iter[dataset] = {}
    for cur_fold in range(5):
        min_error = np.inf
        for r in range(1, 21):
            for lr in [0.01, 0.1, 1.0, 2.0]:
                iter = pd.DataFrame(err[dataset][cur_fold][r][lr]).mean().idxmin()
                error = pd.DataFrame(err[dataset][cur_fold][r][lr]).mean()[iter]
                if error < min_error:
                    min_error = error
                    best_r[dataset][cur_fold] = r
                    best_lr[dataset][cur_fold] = lr
                    best_iter[dataset][cur_fold] = iter

In [6]:
pd.DataFrame(best_r)

,1,2
0,20,11
1,16,13
2,20,17
3,19,15
4,14,14


In [7]:
pd.DataFrame(best_lr)

,1,2
0,2.0,2.0
1,1.0,1.0
2,2.0,1.0
3,1.0,1.0
4,1.0,2.0


In [8]:
pd.DataFrame(best_iter)

,1,2
0,2100,1300
1,500,1300
2,2100,1700
3,500,100
4,500,1700


In [9]:
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from tensor_custom_core import stf_4dim, stf_4dim_time


In [10]:
def mtf(dataset, cur_fold, r=2, lr=1, num_iter=100):
    num_folds=5
    train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
    test_copy = test.copy()
    test_copy[:, 1:, :, :] = np.NaN
    train_test = np.concatenate([train, test_copy])
    H, A, D, T = stf_4dim_time(tensor=train_test, r=r, lr=lr, num_iter=num_iter)
    pred = np.einsum("Hr, Ar, Dr, ATr -> HADT", H, A, D, T)[len(train):, 1:, :, :]
    
    return pred

### dataset1

In [11]:
dataset = 1
out = []
num_fold=5
for cur_fold in range(5):
    b_r= best_r[dataset][cur_fold]
    b_lr = best_lr[dataset][cur_fold]
    b_iter = best_iter[dataset][cur_fold]
    
    pred = mtf(dataset, cur_fold, b_r, b_lr, b_iter)
    
    out.append(pred)

634.205178669 0
377.395198629 50
355.861281052 100
347.117871722 150
343.416226376 200
341.430488927 250
342.437417588 300
339.436308416 350
338.926125741 400
338.551813415 450
338.257340759 500
337.995497989 550
337.746169569 600
337.508540898 650
337.275323372 700
337.042505586 750
336.81027201 800
336.589005076 850
336.377845994 900
336.172773748 950
335.970306953 1000
335.763646105 1050
335.554508584 1100
335.342438693 1150
335.131788962 1200
334.929092296 1250
334.741609053 1300
334.576235819 1350
334.432146603 1400
334.311463552 1450
334.268137686 1500
335.96177768 1550
334.081729034 1600
334.029170781 1650
333.99054924 1700
333.962015966 1750
333.939764955 1800
333.922005968 1850
333.907428952 1900
333.895169444 1950
333.884608368 2000
333.875163209 2050
775.724916684 0
366.570717216 50
350.094776011 100
341.218130973 150
337.954078423 200
336.222316629 250
335.155201852 300
334.437599517 350
333.875780133 400
333.419431722 450
627.18261781 0
371.69572733 50
348.938937458 100
34

In [12]:
cont_out = np.concatenate(out)
tensor = np.load('../2015-5appliances.numpy.npy')
gt = tensor[:, 1:, :, :]

In [13]:
from sklearn.metrics import mean_absolute_error
err_stf = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(cont_out.shape[1])}

In [14]:
err_stf

{'dr': 119.94438254635432,
 'dw': 28.503259073456459,
 'fridge': 39.463749904414321,
 'hvac': 317.73748440484155,
 'mw': 11.658893160385178}

In [15]:
np.save("../code/baseline/mtf-nested-pred-1.npy", cont_out)

In [16]:
np.save("../code/baseline/mtf-nested-error-1.npy", err_stf)

### dataset 2

In [ ]:
dataset = 2
out = []
num_fold=5
for cur_fold in range(5):
    b_r= best_r[dataset][cur_fold]
    b_lr = best_lr[dataset][cur_fold]
    b_iter = best_iter[dataset][cur_fold]
    
    print (b_r, b_lr, b_iter)
    pred = mtf(dataset, cur_fold, b_r, b_lr, b_iter)
    
    out.append(pred)

11 2.0 1300
1127.26288697 0
542.025848806 50
504.793116051 100
492.596434598 150
488.48626187 200
486.411634904 250
485.129892307 300
484.301725783 350
483.729753381 400
483.298527919 450


In [26]:
cont_out_2 = np.concatenate(out)
tensor = np.load('../2015-5appliances-true-agg.npy')
gt = tensor[:, 1:, :, :]

In [27]:
from sklearn.metrics import mean_absolute_error
err_stf = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(cont_out_2.shape[1])}

In [28]:
np.save("../code/baseline/mtf-nested-pred-2.npy", cont_out_2)

In [29]:
np.save("../code/baseline/mtf-nested-error-1.npy", err_stf)

In [30]:
err_stf

{'dr': 119.94438254635432,
 'dw': 28.503259073456459,
 'fridge': 39.463749904414321,
 'hvac': 317.73748440484155,
 'mw': 11.658893160385178}